<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# Converting a From-Scratch GPT Architecture to Llama 2
# 从零开始将 GPT 架构转换为 Llama 2

- In this notebook, we convert the original GPT architecture into a Llama 2 model step by step (note the GPT and GPT-2 share the same architecture)
- 在本笔记本中,我们将原始 GPT 架构一步步转换为 Llama 2 模型(注意 GPT 和 GPT-2 共享相同的架构)

- Why not Llama 1 or Llama 3?
- 为什么不是 Llama 1 或 Llama 3?

   - The Llama 1 architecture is similar to Llama 2, except that Llama 2 has a larger context window (which is nice); the Llama 1 weights are not readily available and have more usage restrictions, so it makes more sense to focus on Llama 2
   - Llama 1 架构与 Llama 2 类似,只是 Llama 2 有更大的上下文窗口(这很好);Llama 1 的权重不容易获得,并且使用限制更多,所以专注于 Llama 2 更有意义

   - Regarding Llama 3, I will share a separate notebook to convert Llama 2 to Llama 3 (there are only a few small additional changes)
   - 关于 Llama 3,我将分享一个单独的笔记本来将 Llama 2 转换为 Llama 3(只有一些小的额外更改)

- The explanations are purposefully kept minimal in this notebook not to bloat it unnecessarily and focus on the main code
- 本笔记本中的解释特意保持最小化,以避免不必要的臃肿并专注于主要代码

- For more information, please see the Llama 2 paper: [Llama 2: Open Foundation and Fine-Tuned Chat Models (2023)](https://arxiv.org/abs/2307.09288)
- 更多信息,请参阅 Llama 2 论文:[Llama 2:开放基础和微调聊天模型(2023)](https://arxiv.org/abs/2307.09288)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/gpt-to-llama/gpt2-to-llama2-llama3.webp?1">

- Packages that are being used in this notebook:
- 本章节中使用的包:

In [1]:
from importlib.metadata import version

pkgs = [
    "huggingface_hub",  # to download pretrained weights
    "sentencepiece",    # to implement the tokenizer
    "torch",            # to implement the model
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

huggingface_hub version: 0.24.7
sentencepiece version: 0.2.0
torch version: 2.4.1+cu121


&nbsp;
# 1. Convert the GPT model implementation step by step
# 1. 逐步转换 GPT 模型实现

- In this section, we go through the GPT model code from [chapter 4](../../ch04/01_main-chapter-code/ch04.ipynb) and modify it step by step to implement the Llama 2 architecture
- 在本节中,我们将逐步修改[第4章](../../ch04/01_main-chapter-code/ch04.ipynb)中的 GPT 模型代码来实现 Llama 2 架构
- Later, we load the original Llama 2 weights shared by Meta AI
- 之后,我们将加载 Meta AI 共享的原始 Llama 2 权重

&nbsp;
## 1.1 Replace LayerNorm with RMSNorm layer
## 1.1 用 RMSNorm 层替换 LayerNorm

- First, we replace LayerNorm by Root Mean Square Layer Normalization (RMSNorm)
- 首先,我们用均方根层归一化(RMSNorm)替换 LayerNorm
- LayerNorm normalizes inputs using mean and variance, while RMSNorm uses only the root mean square, which improves computational efficiency  
- LayerNorm 使用均值和方差来归一化输入,而 RMSNorm 只使用均方根,这提高了计算效率
- The RMSNorm operation is as follows, where $x$ is the input $\gamma$ is a trainable parameter (vector), and $\epsilon$ is a small constant to avoid zero-division errors:
- RMSNorm 操作如下,其中 $x$ 是输入,$\gamma$ 是可训练参数(向量),$\epsilon$ 是一个小常数,用于避免除零错误:

$$y_i = \frac{x_i}{\text{RMS}(x)} \gamma_i, \quad \text{where} \quad \text{RMS}(x) = \sqrt{\epsilon + \frac{1}{n} \sum x_i^2}$$

- For more details, please see the paper [Root Mean Square Layer Normalization (2019)](https://arxiv.org/abs/1910.07467)
- 更多详细信息,请参阅论文 [Root Mean Square Layer Normalization (2019)](https://arxiv.org/abs/1910.07467)

In [2]:
# 导入PyTorch库
import torch
import torch.nn as nn


#####################################
# Chapter 4
#####################################

# class LayerNorm(nn.Module):
#     def __init__(self, emb_dim):
#         super().__init__()
#         self.eps = 1e-5
#         self.scale = nn.Parameter(torch.ones(emb_dim))
#         self.shift = nn.Parameter(torch.zeros(emb_dim))

#     def forward(self, x):
#         mean = x.mean(dim=-1, keepdim=True)
#         var = x.var(dim=-1, keepdim=True, unbiased=False)
#         norm_x = (x - mean) / torch.sqrt(var + self.eps)
#         return self.scale * norm_x + self.shift


# 定义RMSNorm层
class RMSNorm(nn.Module):
    # 初始化函数,接收嵌入维度和epsilon参数
    def __init__(self, emb_dim, eps=1e-5):
        super().__init__()
        self.eps = eps  # 设置epsilon值,避免除零错误
        self.emb_dim = emb_dim  # 保存嵌入维度
        self.weight = nn.Parameter(torch.ones(emb_dim)).float()  # 创建可训练的权重参数

    # 前向传播函数
    def forward(self, x):
        means = x.pow(2).mean(dim=-1, keepdim=True)  # 计算输入的均方值
        x_normed = x * torch.rsqrt(means + self.eps)  # 使用均方根进行归一化
        return (x_normed * self.weight).to(dtype=x.dtype)  # 应用权重并返回结果

- The following code cell checks that this implementation works the same as PyTorch's built-in implementation:
- 下面的代码单元检查此实现与PyTorch的内置实现是否一致:

In [3]:
# 设置随机种子以确保结果可重现
torch.manual_seed(123)

# 创建一个随机张量作为示例输入,形状为(2,3,4)
example_batch = torch.randn(2, 3, 4)

# 实例化我们自定义的RMSNorm层
rms_norm = RMSNorm(emb_dim=example_batch.shape[-1])
# 实例化PyTorch内置的RMSNorm层作为对照
rmsnorm_pytorch = torch.nn.RMSNorm(example_batch.shape[-1], eps=1e-5)

# 验证两个实现的输出结果是否一致
assert torch.allclose(rms_norm(example_batch), rmsnorm_pytorch(example_batch))

&nbsp;
## 1.2 Replace GELU with SiLU activation
## 1.2 将GELU激活函数替换为SiLU激活函数

- Llama uses the SiLU activation function (instead of GELU), which is also known as the Swish function:
- Llama使用SiLU激活函数(代替GELU)，也被称为Swish函数:

$$
\text{silu}(x) = x \cdot \sigma(x), \quad \text{where} \quad \sigma(x) \text{ is the logistic sigmoid.}
$$

- For more information, see the SiLU paper: [Sigmoid-Weighted Linear Units for Neural Network Function Approximation in Reinforcement Learning (2017)](https://arxiv.org/abs/1702.03118)
- 更多信息请参见SiLU论文: [Sigmoid-Weighted Linear Units for Neural Network Function Approximation in Reinforcement Learning (2017)](https://arxiv.org/abs/1702.03118)

In [4]:
#####################################
# Chapter 4
#####################################

# class GELU(nn.Module):
#     def __init__(self):
#         super().__init__()

#     def forward(self, x):
#         return 0.5 * x * (1 + torch.tanh(
#             torch.sqrt(torch.tensor(2.0 / torch.pi)) *
#             (x + 0.044715 * torch.pow(x, 3))
#         ))


# SiLU激活函数类定义
class SiLU(nn.Module):
    # 初始化函数
    def __init__(self):
        super(SiLU, self).__init__()

    # 前向传播函数
    def forward(self, x):
        # 返回输入x与其sigmoid函数值的逐元素乘积
        return x * torch.sigmoid(x)

In [5]:
# 创建SiLU激活函数实例
silu = SiLU()

# 验证自定义SiLU实现与PyTorch内置SiLU函数的输出一致
assert torch.allclose(silu(example_batch), torch.nn.functional.silu(example_batch))

&nbsp;
## 1.3 Update the FeedForward module
## 1.3 更新前馈神经网络模块

- In fact, Llama uses a "Gates Linear Unit" (GLU) variant of SiLU called SwiGLU, which essentially results in a slightly differently structured `FeedForward` module
- 实际上，Llama使用了一种叫做SwiGLU的SiLU门控线性单元(GLU)变体，这本质上导致了`FeedForward`模块结构的略微不同

- SwiGLU uses a gating mechanism in the feedforward layer, with the formula:
- SwiGLU在前馈层中使用门控机制，其公式为:

$$\text{SwiGLU}(x) = \text{SiLU}(\text{Linear}_1(x)) * (\text{Linear}_2(x))$$

- Here, $\text{Linear}_1$ and $\text{Linear}_2$ are two linear layers, and $*$ denotes element-wise multiplication
- 这里，$\text{Linear}_1$和$\text{Linear}_2$是两个线性层，$*$表示逐元素相乘

- The third linear layer, $\text{Linear}_3$, is applied after this gated activation
- 第三个线性层$\text{Linear}_3$在这个门控激活之后应用

- For more information, see SwiGLU paper: [GLU Variants Improve Transformer (2020)](https://arxiv.org/abs/2002.05202)
- 更多信息请参见SwiGLU论文: [GLU Variants Improve Transformer (2020)](https://arxiv.org/abs/2002.05202)

In [6]:
#####################################
# Chapter 4
#####################################
# class FeedForward(nn.Module):
#     def __init__(self, cfg):
#         super().__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
#             GELU(),
#             nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
#         )

#     def forward(self, x):
#         return self.layers(x)

In [7]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        """前馈神经网络模块
        
        参数:
            cfg: 配置字典，包含以下键:
                - emb_dim: 输入维度
                - hidden_dim: 隐藏层维度 
                - dtype: 权重数据类型
        """
        super().__init__()
        self.fc1 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], bias=False)
        self.fc2 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], bias=False)
        self.fc3 = nn.Linear(cfg["hidden_dim"], cfg["emb_dim"], dtype=cfg["dtype"], bias=False)
        self.silu = SiLU()

    def forward(self, x):
        """前向传播
        
        参数:
            x: 输入张量, shape: [batch_size, seq_len, emb_dim]
            
        返回:
            输出张量, shape: [batch_size, seq_len, emb_dim]
        """
        x_fc1 = self.fc1(x)  # 第一个线性变换
        x_fc2 = self.fc2(x)  # 第二个线性变换
        x = self.silu(x_fc1) * x_fc2  # SwiGLU激活
        return self.fc3(x)  # 第三个线性变换

- Note that we also added a `dtype=cfg["dtype"]` setting above, which will allow us to load the model directly in lower precision formats later to save memory (versus instantiating it in the original 32-bit precision format and then converting it)
- 注意我们在上面添加了 `dtype=cfg["dtype"]` 设置,这使我们可以直接以较低精度格式加载模型以节省内存(而不是先以原始32位精度格式实例化然后再转换)

- We also set `bias=False` since Llama doesn't use any bias units
- 我们还设置了 `bias=False`,因为 Llama 不使用任何偏置单元

&nbsp;
## 1.4 Implement RoPE
## 1.4 实现 RoPE

- In the GPT model, the positional embeddings are implemented as follows:
- 在 GPT 模型中,位置嵌入的实现如下:

```python
self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
```
- Instead of these absolute positional embeddings, Llama uses relative positional embeddings, called rotary position embeddings (RoPE for short)
- 与这些绝对位置嵌入不同，Llama使用相对位置嵌入，称为旋转位置嵌入(简称RoPE)
- The reference paper for RoPE is [RoFormer: Enhanced Transformer with Rotary Position Embedding (2021)](https://arxiv.org/abs/2104.09864)
- RoPE的参考论文是[RoFormer: Enhanced Transformer with Rotary Position Embedding (2021)](https://arxiv.org/abs/2104.09864)

In [8]:
def precompute_rope_params(head_dim, theta_base=10_000, context_length=4096):
    """预计算RoPE(旋转位置编码)的参数
    
    参数:
        head_dim: 注意力头的维度
        theta_base: RoPE的基础频率,默认为10000
        context_length: 上下文长度,默认为4096
        
    返回:
        cos, sin: 预计算的余弦和正弦值
    """
    # 确保head_dim是偶数,因为需要成对处理
    assert head_dim % 2 == 0, "Embedding dimension must be even"

    # 计算倒数频率: 1 / (theta_base^(2i/d)),其中i是位置索引,d是head_dim
    inv_freq = 1.0 / (theta_base ** (torch.arange(0, head_dim, 2)[: (head_dim // 2)].float() / head_dim))

    # 生成位置索引: [0, 1, 2, ..., context_length-1]
    positions = torch.arange(context_length)

    # 计算角度: 位置 * 倒数频率,得到形状为(context_length, head_dim//2)的矩阵
    angles = positions[:, None] * inv_freq[None, :]  # Shape: (context_length, head_dim // 2)

    # 将角度扩展到匹配head_dim,通过复制一次实现
    angles = torch.cat([angles, angles], dim=1)  # Shape: (context_length, head_dim)

    # 预计算每个位置的正弦和余弦值
    cos = torch.cos(angles)
    sin = torch.sin(angles)

    return cos, sin

def compute_rope(x, cos, sin):
    """应用RoPE(旋转位置编码)到输入张量
    
    参数:
        x: 输入张量,形状为(batch_size, num_heads, seq_len, head_dim)
        cos: 预计算的余弦值
        sin: 预计算的正弦值
        
    返回:
        经过RoPE变换后的张量
    """
    # 获取输入张量的维度
    batch_size, num_heads, seq_len, head_dim = x.shape
    # 确保head_dim是偶数
    assert head_dim % 2 == 0, "Head dimension must be even"

    # 将输入张量在最后一个维度上分成两半
    x1 = x[..., : head_dim // 2]  # 前半部分
    x2 = x[..., head_dim // 2 :]  # 后半部分

    # 调整sin和cos的形状以匹配输入张量
    cos = cos[:seq_len, :].unsqueeze(0).unsqueeze(0)  # 扩展为(1, 1, seq_len, head_dim)
    sin = sin[:seq_len, :].unsqueeze(0).unsqueeze(0)

    # 应用旋转变换: [x1*cos - x2*sin, x2*cos + x1*sin]
    rotated = torch.cat((-x2, x1), dim=-1)  # 将-x2和x1拼接
    x_rotated = (x * cos) + (rotated * sin)  # 应用旋转

    # 确保输出与输入具有相同的数据类型
    return x_rotated.to(dtype=x.dtype)

- The following is an example of applying RoPE to the `q` and `k` tensors:
- 下面是将RoPE应用于`q`和`k`张量的示例:

In [9]:
# 设置基本参数
# 批次大小为2
batch_size = 2
# 上下文长度为5
context_len = 5  
# 注意力头数为4
num_heads = 4
# 每个头的维度为16
head_dim = 16

# 实例化RoPE参数
# 使用预定义的函数计算余弦和正弦值
cos, sin = precompute_rope_params(head_dim=head_dim, context_length=context_len)

# 创建示例的查询和键张量
# 设置随机种子以保证结果可复现
torch.manual_seed(123)
# 创建随机查询张量,形状为(batch_size, num_heads, context_len, head_dim)
queries = torch.randn(batch_size, num_heads, context_len, head_dim)
# 创建随机键张量,形状与查询张量相同
keys = torch.randn(batch_size, num_heads, context_len, head_dim)

# 应用旋转位置编码
# 对查询张量应用RoPE变换
queries_rot = compute_rope(queries, cos, sin)
# 对键张量应用RoPE变换
keys_rot = compute_rope(keys, cos, sin)

&nbsp;
## 1.5 Add RoPE to MultiHeadAttention module
## 1.5 为MultiHeadAttention模块添加RoPE

- It's important to note that GPT applies the positional embeddings to the inputs, whereas Llama applies rotations to the query and key vectors in the self-attention mechanism itself
- 需要注意的是，GPT将位置嵌入应用于输入，而Llama在自注意力机制中对查询和键向量应用旋转

- Here, we modify the `MultiHeadAttention` class with the appropriate RoPE code
- 在这里，我们使用适当的RoPE代码修改`MultiHeadAttention`类

- In addition, we remove the `qkv_bias` option and hardcode the `bias=False` setting
- 此外，我们移除了`qkv_bias`选项并将`bias=False`设置硬编码

- Also, we add a dtype setting to be able to instantiate the model with a lower precision later
- 同时，我们添加了dtype设置，以便稍后能够以较低精度实例化模型

- Tip: since the `TransformerBlock`s (in the next section) are repeated exactly, we could simplify the code and only initialize the buffers once instead for each `MultiHeadAttention` module; however, we add the precomputed RoPE parameters to the `MultiHeadAttention` class so that it can function as a standalone module
- 提示：由于`TransformerBlock`（在下一节中）是完全重复的，我们可以简化代码，只为每个`MultiHeadAttention`模块初始化一次缓冲区；但是，我们将预计算的RoPE参数添加到`MultiHeadAttention`类中，使其能够作为独立模块运行

In [10]:
#####################################
# 第3章
#####################################
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, num_heads, dtype=None):  # ,dropout, num_heads, qkv_bias=False):
        # 继承父类初始化
        super().__init__()
        # 确保输出维度可以被注意力头数整除
        assert d_out % num_heads == 0, "d_out must be divisible by n_heads"

        # 保存输出维度、注意力头数和每个头的维度
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # 将投影维度减小以匹配所需的输出维度

        ################################### NEW ###################################
        # 为所有线性层设置bias=False和dtype=dtype
        ###########################################################################
        # 创建查询、键、值的线性变换层
        self.W_query = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.W_key = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.W_value = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.out_proj = nn.Linear(d_out, d_out, bias=False, dtype=dtype)  # 用于组合头部输出的线性层
        # self.dropout = nn.Dropout(dropout)
        
        # 注册因果掩码作为缓冲区
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

        ################################### NEW ###################################
        # 预计算RoPE参数并注册为缓冲区
        cos, sin = precompute_rope_params(head_dim=self.head_dim, context_length=context_length)
        self.register_buffer("cos", cos)
        self.register_buffer("sin", sin)
        ###########################################################################


    def forward(self, x):
        # 获取输入张量的维度
        b, num_tokens, d_in = x.shape

        # 通过线性层计算键、查询和值
        keys = self.W_key(x)  # 形状: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # 重塑张量以添加num_heads维度
        # 展开最后一个维度: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # 转置维度: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        ################################### NEW ###################################
        # 对键和查询应用RoPE变换
        keys = compute_rope(keys, self.cos, self.sin)
        queries = compute_rope(queries, self.cos, self.sin)
        ###########################################################################

        # 使用因果掩码计算缩放点积注意力(即自注意力)
        attn_scores = queries @ keys.transpose(2, 3)  # 计算每个头的点积

        # 将原始掩码截断到token数量并转换为布尔值
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # 使用掩码填充注意力分数
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        # 计算注意力权重
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        # attn_weights = self.dropout(attn_weights)

        # 计算上下文向量 形状: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # 组合所有头部，其中self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.reshape(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # 可选的投影层

        return context_vec

- Below is an example using the `MultiHeadAttention` module on an example input:
- 下面是一个使用`MultiHeadAttention`模块的输入示例:

In [11]:
# 设置参数
batch_size = 1  # 批次大小
context_len = 100  # 上下文长度
max_context_len = 4096  # 最大上下文长度
embed_dim = 128  # 嵌入维度
num_heads = 4  # 注意力头数

# 创建一个随机输入张量作为示例
example_batch = torch.randn((batch_size, context_len, embed_dim))

# 初始化多头注意力模块
mha = MultiHeadAttention(
    d_in=embed_dim,  # 输入维度
    d_out=embed_dim,  # 输出维度
    context_length=max_context_len,  # 最大上下文长度
    num_heads=num_heads  # 注意力头数
)

# 对示例输入进行前向传播
mha(example_batch)

del mha  # 删除模型以释放内存

&nbsp;
## 1.6 Update the TransformerBlock module
## 1.6 更新 TransformerBlock 模块

- At this stage, most of the hard work is already done; we can now update the `TransformerBlock` to use the code we implemented above
- 在这个阶段,大部分困难的工作已经完成;我们现在可以更新`TransformerBlock`来使用我们上面实现的代码
- This means we
- 这意味着我们需要:
 - replace LayerNorm with RMSNorm
 - 用RMSNorm替换LayerNorm
 - remove dropout
 - 移除dropout
 - remove the `qkv_bias` setting
 - 移除`qkv_bias`设置
 - add the `dtype` setting
 - 添加`dtype`设置

In [12]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        # 继承自nn.Module基类
        super().__init__()
        
        # 初始化多头注意力层
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],      # 输入维度
            d_out=cfg["emb_dim"],     # 输出维度
            context_length=cfg["context_length"],  # 上下文长度
            num_heads=cfg["n_heads"],  # 注意力头数
            dtype=cfg["dtype"]         # 数据类型
            # 移除dropout和qkv_bias参数
        )
        
        # 初始化前馈神经网络层
        self.ff = FeedForward(cfg)

        # 使用RMSNorm替代LayerNorm进行归一化
        self.norm1 = RMSNorm(cfg["emb_dim"])  # 第一个归一化层
        self.norm2 = RMSNorm(cfg["emb_dim"])  # 第二个归一化层
        
        # 移除dropout层

    def forward(self, x):
        # 注意力模块的残差连接
        shortcut = x                   # 保存输入用于残差连接
        x = self.norm1(x)             # 第一次归一化
        x = self.att(x)               # 通过多头注意力层,输出形状为[batch_size, num_tokens, emb_size]
        x = x + shortcut              # 添加残差连接

        # 前馈网络模块的残差连接
        shortcut = x                   # 保存输入用于残差连接
        x = self.norm2(x)             # 第二次归一化
        x = self.ff(x)                # 通过前馈神经网络
        x = x + shortcut              # 添加残差连接

        return x                       # 返回最终输出

&nbsp;
## 1.7 Update the model class
## 1.7 更新模型类

- As you may recall from [chapter 5](../01_main-chapter-code/ch05.ipynb), the `TransformerBlock` is a repeated block within the main model
- 回顾第5章，`TransformerBlock`是主模型中的一个重复块

- Our Llama model is almost complete; we just have to update the model code surrounding the `TransformerBlock`
- 我们的Llama模型即将完成，只需要更新`TransformerBlock`周围的模型代码

- This means we
- 这意味着我们需要：

  - remove absolute positional embeddings since we have RoPE embeddings now
  - 由于现在使用了RoPE嵌入，移除绝对位置嵌入
  
  - replace LayerNorm with RMSNorm
  - 用RMSNorm替换LayerNorm
  
  - remove dropout
  - 移除dropout层
  
  - add the dtype setting
  - 添加dtype设置

In [13]:
# 将GPT模型类改为Llama2模型类
class Llama2Model(nn.Module):
    def __init__(self, cfg):
        # 初始化父类
        super().__init__()
        
        # 创建词嵌入层,将词表大小映射到嵌入维度,指定数据类型
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"], dtype=cfg["dtype"])
        
        # 注释掉位置嵌入,因为使用RoPE代替
        # self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        # 注释掉dropout层,Llama2不使用
        # self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # 创建transformer块的序列,重复n_layers次
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        # 将最后的LayerNorm替换为RMSNorm
        # self.final_norm = LayerNorm(cfg["emb_dim"]) 
        self.final_norm = RMSNorm(cfg["emb_dim"])
        
        # 创建输出层,将嵌入维度映射回词表大小,不使用偏置项,指定数据类型
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False, dtype=cfg["dtype"])

    def forward(self, in_idx):
        # 获取输入的batch大小和序列长度
        batch_size, seq_len = in_idx.shape
        
        # 通过词嵌入层获取词向量
        tok_embeds = self.tok_emb(in_idx)
        
        # 注释掉位置嵌入的计算
        # pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        
        # 直接使用词嵌入作为输入,不再加入位置嵌入
        x = tok_embeds  # + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        
        # 注释掉dropout
        # x = self.drop_emb(x)
        
        # 依次通过transformer块
        x = self.trf_blocks(x)
        
        # 通过最终的RMSNorm层
        x = self.final_norm(x)
        
        # 通过输出层得到logits
        logits = self.out_head(x)
        
        # 返回预测结果
        return logits

&nbsp;
## 2. Initialize model
## 2. 初始化模型

- The model code is now complete, and we are ready to initialize it
- 模型代码现已完成，我们可以开始初始化它
- In [chapter 5](../01_main-chapter-code/ch05.ipynb), we used the following config file to specify the 124M-parameter GPT model:
- 在[第5章](../01_main-chapter-code/ch05.ipynb)中，我们使用了以下配置文件来指定124M参数的GPT模型：

In [14]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,     # 词表大小
    "context_length": 1024,  # 上下文长度
    "emb_dim": 768,          # 嵌入维度
    "n_heads": 12,           # 注意力头数量
    "n_layers": 12,          # 层数
    "drop_rate": 0.1,        # Dropout比率
    "qkv_bias": False        # 查询-键-值偏置项
}

- For reference, the 1.5B parameter GPT model config is shown below as well:
- 作为参考，下面也展示了15亿参数的GPT模型配置:

In [15]:
GPT_CONFIG_1558M = {
    "vocab_size": 50257,     # Vocabulary size 词表大小
    "context_length": 1024,  # Context length 上下文长度
    "emb_dim": 1600,         # Embedding dimension 嵌入维度
    "n_heads": 25,           # Number of attention heads 注意力头数量
    "n_layers": 48,          # Number of layers 层数
    "drop_rate": 0.1,        # Dropout rate Dropout比率
    "qkv_bias": False        # Query-Key-Value bias 查询-键-值偏置项
}

- Similarly, we can define a Llama 2 config file for the 7B model (we ignore the other larger models for simplicity here):
- 类似地，我们可以为7B模型定义一个Llama 2配置文件（为了简单起见，这里我们忽略其他更大的模型）：

In [16]:
LLAMA2_CONFIG_7B = {
    "vocab_size": 32000,     # 词表大小
    "context_length": 4096,  # 上下文长度
    "emb_dim": 4096,         # 嵌入维度
    "n_heads": 32,           # 注意力头数量
    "n_layers": 32,          # 层数
    "hidden_dim": 11008,     # 新增：前馈网络中的中间维度大小
    "dtype": torch.bfloat16  # 新增：使用低精度数据类型以节省内存
}

- Using these settings, we can now initialize a Llama 2 7B model (note that this requires ~26 GB of memory)
- 使用这些设置，我们现在可以初始化一个Llama 2 7B模型（注意这需要约26 GB内存）

In [17]:
# 使用Llama2配置初始化一个7B参数的模型
model = Llama2Model(LLAMA2_CONFIG_7B)

In [18]:
# 计算模型的总参数量
total_params = sum(p.numel() for p in model.parameters())
# 打印总参数量
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 6,738,415,616


- As shown above, the model contains 6.7 billion parameters (commonly rounded and referred to as a 7B model)
- 如上所示，该模型包含67亿个参数（通常四舍五入并称为7B模型）
- Additionally, we can calculate the memory requirements for this model using the code below:
- 此外，我们可以使用以下代码计算该模型的内存需求：

In [19]:
# 定义一个函数来计算模型的内存占用大小
def model_memory_size(model, input_dtype=torch.float32):
    # 初始化参数总数为0
    total_params = 0
    # 初始化梯度总数为0 
    total_grads = 0
    # 遍历模型的所有参数
    for param in model.parameters():
        # 计算每个参数的元素总数
        param_size = param.numel()
        # 累加到参数总数中
        total_params += param_size
        # 检查该参数是否需要存储梯度
        if param.requires_grad:
            # 如果需要梯度,累加到梯度总数中
            total_grads += param_size

    # 计算缓冲区大小(需要内存的非参数部分)
    total_buffers = sum(buf.numel() for buf in model.buffers())

    # 计算每个元素的字节大小
    element_size = torch.tensor(0, dtype=input_dtype).element_size()
    # 计算总内存字节数 = (元素总数) * (每个元素的字节大小)
    total_memory_bytes = (total_params + total_grads + total_buffers) * element_size

    # 将字节转换为GB
    total_memory_gb = total_memory_bytes / (1024**3)

    # 返回总内存大小(GB)
    return total_memory_gb

# 打印使用float32数据类型时的内存占用
print(f"float32 (PyTorch default): {model_memory_size(model, input_dtype=torch.float32):.2f} GB")
# 打印使用bfloat16数据类型时的内存占用
print(f"bfloat16: {model_memory_size(model, input_dtype=torch.bfloat16):.2f} GB")

float32 (PyTorch default): 52.33 GB
bfloat16: 26.17 GB


- Lastly, we can also transfer the model to an NVIDIA or Apple Silicon GPU if applicable:
- 最后，如果可用的话，我们还可以将模型转移到 NVIDIA 或 Apple Silicon GPU 上:

In [20]:
# 检查是否有CUDA GPU可用
if torch.cuda.is_available():
    device = torch.device("cuda")
# 检查是否有Apple Silicon GPU (MPS)可用    
elif torch.backends.mps.is_available():
    device = torch.device("mps") 
# 如果都不可用则使用CPU
else:
    device = torch.device("cpu")

# 将模型移动到选定的设备上
model.to(device);

&nbsp;
## 3. Load tokenizer
## 3. 加载分词器

- In this section, we are going to load the tokenizer for the model
- 在本节中,我们将为模型加载分词器
- Llama 2 uses Google's [SentencePiece](https://github.com/google/sentencepiece) tokenizer instead of OpenAI's [Tiktoken](https://github.com/openai/tiktoken) (but Llama 3 uses Tiktoken)
- Llama 2 使用 Google 的 [SentencePiece](https://github.com/google/sentencepiece) 分词器而不是 OpenAI 的 [Tiktoken](https://github.com/openai/tiktoken) (但 Llama 3 使用 Tiktoken)
- Meta AI shared the original Llama 2 model weights and tokenizer vocabulary on the Hugging Face Hub
- Meta AI 在 Hugging Face Hub 上共享了原始的 Llama 2 模型权重和分词器词汇表
- We will download the tokenizer vocabulary from the Hub and load it into SentencePiece
- 我们将从 Hub 下载分词器词汇表并将其加载到 SentencePiece 中
- Uncomment and run the following code to install the required libraries:
- 取消注释并运行以下代码以安装所需的库:

In [21]:
# !pip install huggingface_hub sentencepiece

- Please note that Meta AI requires that you accept the Llama 2 licensing terms before you can download the files; to do this, you have to create a Hugging Face Hub account and visit the [meta-llama/Llama-2-7b](https://huggingface.co/meta-llama/Llama-2-7b) repository to accept the terms
- 请注意，Meta AI 要求您在下载文件之前接受 Llama 2 许可条款；为此，您需要创建一个 Hugging Face Hub 账户并访问 [meta-llama/Llama-2-7b](https://huggingface.co/meta-llama/Llama-2-7b) 仓库来接受条款

- Next, you will need to create an access token; to generate an access token with READ permissions, click on the profile picture in the upper right and click on "Settings"
- 接下来，您需要创建一个访问令牌；要生成具有读取权限的访问令牌，请点击右上角的个人头像，然后点击"Settings"（设置）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/gpt-to-llama/settings.webp?1" width="300px">

- Then, create and copy the access token so you can copy & paste it into the next code cell
- 然后，创建并复制访问令牌，以便您可以将其复制并粘贴到下一个代码单元格中

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/gpt-to-llama/access-token.webp?1" width="600px">

In [22]:
# 从 huggingface_hub 导入 login 函数
from huggingface_hub import login
# 导入 json 模块
import json

# 打开并读取配置文件
with open("config.json", "r") as config_file:
    # 加载 JSON 配置
    config = json.load(config_file)
    # 从配置中获取访问令牌
    access_token = config["HF_ACCESS_TOKEN"]

# 使用访问令牌登录 Hugging Face Hub
login(token=access_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


- After login via the access token, which is necessary to verify that we accepted the Llama 2 licensing terms, we can now download the tokenizer vocabulary:
- 通过访问令牌登录后，这是验证我们已接受 Llama 2 许可条款所必需的，现在我们可以下载分词器词汇表：

In [23]:
# 从 huggingface_hub 导入 hf_hub_download 函数
from huggingface_hub import hf_hub_download

# 下载分词器文件
tokenizer_file = hf_hub_download(
    repo_id="meta-llama/Llama-2-7b",    # 模型仓库 ID
    filename="tokenizer.model",          # 分词器文件名
    local_dir="Llama-2-7b"              # 本地保存目录
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

- To provide a more familiar interface for the tokenizer, we define a small `LlamaTokenizer` wrapper class:
- 为了提供一个更熟悉的分词器接口，我们定义一个小型的 `LlamaTokenizer` 包装类：

In [24]:
# 导入 sentencepiece 库并重命名为 spm
import sentencepiece as spm


# 定义 LlamaTokenizer 类用于封装分词器功能
class LlamaTokenizer:
    # 初始化方法，接收分词器文件路径作为参数
    def __init__(self, filepath):
        # 创建 SentencePieceProcessor 实例
        sp = spm.SentencePieceProcessor()
        # 加载分词器模型文件
        sp.load(tokenizer_file)
        # 将分词器实例保存为类属性
        self.tokenizer = sp

    # 将文本编码为 token ID 序列的方法
    def encode(self, text):
        # 调用分词器的 encode_as_ids 方法进行编码
        return self.tokenizer.encode_as_ids(text)

    # 将 token ID 序列解码为文本的方法
    def decode(self, ids):
        # 调用分词器的 decode_pieces 方法进行解码
        return self.tokenizer.decode_pieces(ids)


# 使用分词器文件创建 LlamaTokenizer 实例
tokenizer = LlamaTokenizer(tokenizer_file)

- We can now use the `generate` function to have the Llama 2 model generate new text:
- 现在我们可以使用 `generate` 函数让 Llama 2 模型生成新的文本：

In [25]:
# 从前面章节导入所需函数
from previous_chapters import generate, text_to_token_ids, token_ids_to_text


# 设置随机种子以保证结果可复现
torch.manual_seed(123)

# 生成文本
token_ids = generate(
    model=model,  # 使用加载的模型
    idx=text_to_token_ids("Every effort moves", tokenizer).to(device),  # 将输入文本转换为token ID并移至设备
    max_new_tokens=30,  # 最多生成30个新token
    context_size=LLAMA2_CONFIG_7B["context_length"],  # 使用Llama 2配置中的上下文长度
    top_k=1,  # 只选择概率最高的token
    temperature=0.  # 温度为0,使输出确定性
)

# 将生成的token ID转换回文本并打印
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort movesαllRadius deletingpretcc否']; future eer napulate lackус während inter DES издаSchéon로жа Bass differencespadxsnu ;; ctx始


 - Of course, as we can see above, the text is nonsensical since we haven't trained the Llama 2 model yet
 - 当然,正如我们在上面看到的,由于我们还没有训练 Llama 2 模型,生成的文本是没有意义的
 - In the next section, instead of training it ourselves, which would cost tens to hundreds of thousands of dollars, we load the pretrained weights from Meta AI
 - 在下一节中,我们不会自己训练模型(这将花费数十万到数十万美元),而是加载来自 Meta AI 的预训练权重

&nbsp;
## 4. Load pretrained weights
## 4. 加载预训练权重

- We are loading the ["meta-llama/Llama-2-7b"](https://huggingface.co/meta-llama/Llama-2-7b) base model below, which is a simple text completion model before finetuning
- 我们将在下面加载 ["meta-llama/Llama-2-7b"](https://huggingface.co/meta-llama/Llama-2-7b) 基础模型，这是一个在微调之前的简单文本补全模型
- Alternatively, you can load the instruction-finetuned and aligned ["meta-llama/Llama-2-7b-chat"](https://huggingface.co/meta-llama/Llama-2-7b-chat) model by modifying the string in the next code cell accordingly
- 或者，您可以通过修改下一个代码单元中的字符串来加载经过指令微调和对齐的 ["meta-llama/Llama-2-7b-chat"](https://huggingface.co/meta-llama/Llama-2-7b-chat) 模型

In [26]:
# 从 Hugging Face Hub 下载 Llama-2-7b 模型权重文件
# repo_id: 模型仓库ID
# filename: 权重文件名
# local_dir: 本地保存目录
weights_file = hf_hub_download(
   repo_id="meta-llama/Llama-2-7b",
   filename="consolidated.00.pth", 
   local_dir="Llama-2-7b"
)

consolidated.00.pth:   0%|          | 0.00/13.5G [00:00<?, ?B/s]

In [27]:
# 加载模型权重文件
# weights_only=True 表示只加载权重参数,不加载优化器状态等其他信息
weights = torch.load(weights_file, weights_only=True)

- The `weights` contains the following tensors (only the first 15 are shown for simplicity):
- `weights` 包含以下张量(为简单起见,仅显示前 15 个):

In [28]:
# 列出权重字典中的前15个键
list(weights.keys())[:15]

['tok_embeddings.weight',
 'norm.weight',
 'output.weight',
 'layers.0.attention.wq.weight',
 'layers.0.attention.wk.weight',
 'layers.0.attention.wv.weight',
 'layers.0.attention.wo.weight',
 'layers.0.feed_forward.w1.weight',
 'layers.0.feed_forward.w2.weight',
 'layers.0.feed_forward.w3.weight',
 'layers.0.attention_norm.weight',
 'layers.0.ffn_norm.weight',
 'layers.1.attention.wq.weight',
 'layers.1.attention.wk.weight',
 'layers.1.attention.wv.weight']

- The following function, modeled after the `load_weights_into_gpt` function in [chapter 5](../01_main-chapter-code/ch05.ipynb), loads the pretrained weights into our Llama 2 model:
- 以下函数参考了[第5章](../01_main-chapter-code/ch05.ipynb)中的`load_weights_into_gpt`函数，将预训练权重加载到我们的 Llama 2 模型中：

In [29]:
# 定义一个辅助函数,用于将权重参数赋值给模型
def assign(left, right):
    # 检查左右两个张量的形状是否匹配
    if left.shape != right.shape:
        raise ValueError(f"Shape mismatch. Left: {left.shape}, Right: {right.shape}")

    # 如果右侧是张量,则创建一个新的参数对象
    if isinstance(right, torch.Tensor):
        return torch.nn.Parameter(right.clone().detach())
    # 如果右侧不是张量,则先转换为张量再创建参数对象
    else:
        return torch.nn.Parameter(torch.tensor(right))


# 定义加载预训练权重到Llama模型的主函数
def load_weights_into_llama(model, param_config, params):
    # 加载词嵌入层权重
    model.tok_emb.weight = assign(model.tok_emb.weight, params["tok_embeddings.weight"])

    # 遍历每一个transformer层
    for l in range(param_config["n_layers"]):

        # 加载注意力层的权重
        # 加载查询矩阵权重
        model.trf_blocks[l].att.W_query.weight = assign(
            model.trf_blocks[l].att.W_query.weight,
            params[f"layers.{l}.attention.wq.weight"]
        )
        # 加载键矩阵权重
        model.trf_blocks[l].att.W_key.weight = assign(
            model.trf_blocks[l].att.W_key.weight,
            params[f"layers.{l}.attention.wk.weight"]
        )
        # 加载值矩阵权重
        model.trf_blocks[l].att.W_value.weight = assign(
            model.trf_blocks[l].att.W_value.weight,
            params[f"layers.{l}.attention.wv.weight"]
        )
        # 加载输出投影层权重
        model.trf_blocks[l].att.out_proj.weight = assign(
            model.trf_blocks[l].att.out_proj.weight,
            params[f"layers.{l}.attention.wo.weight"]
        )
        # 加载第一个层归一化权重
        model.trf_blocks[l].norm1.weight = assign(
            model.trf_blocks[l].norm1.weight,
            params[f"layers.{l}.attention_norm.weight"]
        )

        # 加载前馈网络层的权重
        # 加载第一个全连接层权重
        model.trf_blocks[l].ff.fc1.weight = assign(
            model.trf_blocks[l].ff.fc1.weight,
            params[f"layers.{l}.feed_forward.w1.weight"]
        )
        # 注意:w2和w3在权重文件中的顺序是相反的
        # 加载第二个全连接层权重(使用w3)
        model.trf_blocks[l].ff.fc2.weight = assign(
            model.trf_blocks[l].ff.fc2.weight,
            params[f"layers.{l}.feed_forward.w3.weight"]
        )
        # 加载第三个全连接层权重(使用w2)
        model.trf_blocks[l].ff.fc3.weight = assign(
            model.trf_blocks[l].ff.fc3.weight,
            params[f"layers.{l}.feed_forward.w2.weight"]
        )
        # 加载第二个层归一化权重
        model.trf_blocks[l].norm2.weight = assign(
            model.trf_blocks[l].norm2.weight,
            params[f"layers.{l}.ffn_norm.weight"]
        )

    # 加载最终输出层的权重
    # 加载最后的层归一化权重
    model.final_norm.weight = assign(model.final_norm.weight, params["norm.weight"])
    # 加载输出头层权重
    model.out_head.weight = assign(model.out_head.weight, params["output.weight"])


# 使用上述函数加载权重到模型中
load_weights_into_llama(model, LLAMA2_CONFIG_7B, weights)
# 将模型移动到指定设备(CPU/GPU)
model.to(device);

- Next, we are ready to use the model for text generation
- 接下来，我们准备使用模型进行文本生成

In [30]:
# 设置随机种子以确保可重复性
torch.manual_seed(123)

# 生成文本
token_ids = generate(
    model=model,  # 使用加载的模型
    idx=text_to_token_ids("Every effort", tokenizer).to(device),  # 将输入文本转换为token并移至设备
    max_new_tokens=25,  # 最多生成25个新token
    context_size=LLAMA2_CONFIG_7B["context_length"],  # 使用配置中定义的上下文长度
    top_k=1,  # 只选择概率最高的token
    temperature=0.  # 温度为0,使输出更确定性
)

# 将生成的token转换回文本并打印
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort has been made to ensure that the information contained in this website is accurate and up to date and correct at the time of publication


&nbsp;
## 5. Using the instruction-finetuned model
## 5. 使用指令微调模型

- As mentioned earlier, above we used the pretrained base model; if you want to use a model capable of following instructions, use the `"meta-llama/Llama-2-7b-chat"` model instead, as shown below
- 如前所述，上面我们使用了预训练的基础模型；如果你想使用一个能够遵循指令的模型，请改用`"meta-llama/Llama-2-7b-chat"`模型，如下所示

In [34]:
# 删除模型以释放内存
del model  # to free up memory

# 从Hugging Face Hub下载Llama-2-7b-chat模型权重文件
weights_file = hf_hub_download(
   repo_id="meta-llama/Llama-2-7b-chat",
   filename="consolidated.00.pth", 
   local_dir="Llama-2-7b-chat"
)

# 创建新的Llama2模型实例
model = Llama2Model(LLAMA2_CONFIG_7B)
# 将权重加载到模型中
load_weights_into_llama(model, LLAMA2_CONFIG_7B, weights)
# 将模型移动到指定设备
model.to(device);

# 设置随机种子以确保可重复性
torch.manual_seed(123)

# 使用模型生成文本
token_ids = generate(
    model=model,  # 使用加载的模型
    idx=text_to_token_ids("What do llamas eat?", tokenizer).to(device),  # 将输入问题转换为token并移至设备
    max_new_tokens=25,  # 最多生成25个新token
    context_size=LLAMA2_CONFIG_7B["context_length"],  # 使用配置中定义的上下文长度
    top_k=1,  # 只选择概率最高的token
    temperature=0.  # 温度为0,使输出更确定性
)

# 将生成的token转换回文本并打印输出
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

consolidated.00.pth:   0%|          | 0.00/13.5G [00:00<?, ?B/s]

Output text:
 What do llamas eat?
Llamas and alpacas are herbivores, which means they eat grasses, leaves, grass


&nbsp;
# What's next?
# 接下来是什么？

- This notebook converted the original GPT-2 architecture into a Llama 2 model
- 本笔记本将原始的GPT-2架构转换为Llama 2模型
- If you are interested in how to convert Llama 2 into Llama 3, Llama 3.1, and Llama 3.2, check out the [converting-llama2-to-llama3.ipynb](converting-llama2-to-llama3.ipynb) notebook
- 如果你对如何将Llama 2转换为Llama 3、Llama 3.1和Llama 3.2感兴趣，请查看[converting-llama2-to-llama3.ipynb](converting-llama2-to-llama3.ipynb)笔记本